In [1]:
import numpy as np
from hftbacktest.data.utils import databento

In [2]:
import os 
base_path = os.getcwd()
path = os.path.join(base_path, 'data/mbp.parquet')
data = databento.convert(path)

In [ ]:
data = data[:100]

In [ ]:

np.savez('test_dbn', data=data)

In [ ]:
from hftbacktest.data.utils import create_last_snapshot


data = create_last_snapshot("/Users/danny/projects/hftbacktest/test_dbn.npz", tick_size=0.01, lot_size=0.001)
np.savez(path + 'bn_eod.npz', data=data)

# create_last_snapshot(
#     '/Users/coopergamble/Desktop/usc/clubs/quant/hftbacktest/test_dbn.npz',
#     tick_size=0.01,
#     lot_size=0.001,
#     initial_snapshot='btcusdt_20230404_eod.npz',
#     output_snapshot_filename='btcusdt_20230405_eod'
# )